In [51]:
OLD_DATASET = 'therealvul/parlertts_pony_speech_tagged_stage3'
NEW_TAGS = 'therealvul/parlertts_pony_speech_tags_stage2'
NEW_DATASET = OLD_DATASET

from datasets import load_dataset

old_data = load_dataset(OLD_DATASET, split='train').to_pandas()
new_tags = load_dataset(NEW_TAGS, split='train').to_pandas()

In [52]:
from datasets import Dataset
# Merge on new tags
merged_df = new_tags.merge(
    old_data[['speaker', 'source', 'start', 'text_description']], 
    on=['speaker', 'source', 'start'], how='left')
merged_dataset = Dataset.from_pandas(merged_df, preserve_index=False)

In [53]:
print(len(merged_dataset))
missing_rows = merged_dataset.filter(
    lambda ex: ex['text_description'] is None, num_proc=4)
print(len(missing_rows))

64780


Filter (num_proc=4):   0%|          | 0/64780 [00:00<?, ? examples/s]

836


In [54]:
NEW_SINGLE_SPEAKER_PROMPT = """You will be given four descriptive keywords related to an audio sample of [speaker_name]'s speech. These keywords include:
1. The level of reverberation (very distant-sounding, distant-sounding, slightly distant-sounding, slightly close-sounding, very close-sounding)
3. The amount of noise in the sample (extremely noisy, very noisy, noisy, slightly noisy, almost no noise, very clear)
3. The tone of the speaker's voice (very monotone, monotone, slightly expressive and animated, expressive and animated, very expressive and animated)
4. The pace of the speaker's delivery (very slowly, slowly, slightly slowly, moderate speed, slightly fast, fast, very fast)

Your task is to create a text description using these keywords that accurately describes [speaker_name]'s speech sample.
If the amount of noise is 'very noisy' and the level of reverberation is 'very distant-sounding', you must include terms such as 'very poor recording' or `very bad recording` in the description. 
Likewise, if the amount of noise is 'very clear' and the level of reverberation is 'very close-sounding', you must include terms like 'very good recording' or `excellent recording` in the description. 
You can randomly omit the following terms, as they are default terms: 'moderate speed' and 'moderate pitch'.
Do not add extra details beyond what has been provided above. You can change the order of keywords, and replace synonymous terms.

For example, given the following keywords: 'slightly distant-sounding', 'clear', 'very expressive and animated', 'slightly fast', a valid description would be: '[speaker_name] speaks slightly fast but has a very animated delivery in a room with slight echo but no background noise.'
Another valid description would be: `In a very animated voice, [speaker_name] delivers words slightly quickly. The room is quite, but there's a bit of echo.'

Ensure that the generated description is grammatically correct, easy to understand, and concise. Only return one and only one description.

For the keywords: ''[reverberation]', '[sdr_noise]', '[speech_monotony]', '[speaking_rate]', the corresponding description is:
"""

from transformers import AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
INSTRUCT_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(
    INSTRUCT_MODEL,
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype = torch.float16,
        bnb_4bit_quant_type = 'nf4',
        bnb_4bit_use_double_quant = False
    ),
    max_length = 2048,
    device_map='auto')
chatbot = pipeline("text-generation", model=model,
    tokenizer=INSTRUCT_MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [55]:
def generate_text_description(example):
    if example['text_description'] is not None:
        return example
    print(example['text_description'])
    keys = ['reverberation', 'sdr_noise', 'speech_monotony',
        'speaking_rate']
    prompt = NEW_SINGLE_SPEAKER_PROMPT.replace(
        '[speaker_name]', example['speaker'])
    for key in keys:
        prompt = prompt.replace(
            f'[{key}]', example[key])
    messages = [{'role': 'user', 'content': prompt}]
    response = chatbot(messages)
    messages = response[0]['generated_text']
    example['text_description'] = messages[1]['content']
    print(example['text_description'])
    return example

updated_dataset = merged_dataset.map(generate_text_description)

Map:   0%|          | 0/64780 [00:00<?, ? examples/s]

c:\Users\vul\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1376: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Applejack's speech is delivered with a high level of expressiveness and animation, yet the recording quality is slightly poor due to a moderate amount of background noise and a slight echo, giving the impression of a distant sound.
None


...

 Fluttershy delivers her words with great expressiveness and animation, yet the recording is slightly marred by a touch of background noise. The close proximity of the recording gives the impression of an intimate conversation.
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 "Sludge's speech is slightly expressive and animated, yet the recording quality is poor due to a high level of background noise and a distant, echoing sound."
None


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 "Rumble's voice is very expressive and animated, yet the recording quality is poor due to a high level of noise and a distant, echoing sound."
None
 Fluttershy's speech is slightly slowly delivered with a slightly expressive and animated tone, yet the recording quality is poor due to a noticeable amount of background noise and a slight echo, giving the impression of a distant sound.


In [58]:
updated_dataset.push_to_hub('therealvul/parlertts_pony_speech_tagged_stage3')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/65 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/therealvul/parlertts_pony_speech_tagged_stage3/commit/1802f81ecfb92f2e6e5dc6502019c9b0caa40aa9', commit_message='Upload dataset', commit_description='', oid='1802f81ecfb92f2e6e5dc6502019c9b0caa40aa9', pr_url=None, pr_revision=None, pr_num=None)